In [10]:
%pip install langchain langchain-google-genai langchain_community tavily-python aiosqlite langchain-tavily 
%pip install langgraph-checkpoint-sqlite 
%pip install langgraph langgraph-checkpoint-sqlite
%pip install langgraph 
%pip install tavily-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\djfar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\djfar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\djfar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\djfar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\djfar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_tavily import TavilySearch
import sqlite3 
from langgraph.checkpoint.sqlite import SqliteSaver 
conn = sqlite3.connect("checkpoints.db", check_same_thread=False) 
memory = SqliteSaver(conn)

In [ ]:
import google.generativeai as genai
import os
import operator
import uuid
import sqlite3
from typing import TypedDict, Annotated, List

# Dependências do LangChain/LangGraph
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver

# --- Configuração Inicial ---

# Simulação de Carregamento de Chaves (Assumindo que o usuário tem um .env)
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
# Nota: As chaves devem estar configuradas no ambiente onde o código será executado.

# Inicialização do LLM (Gemini 2.5 Flash)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

# Inicialização da Ferramenta de Busca (Tavily)
# Simulação das políticas de comunidade para o Tavily pesquisar
# Nota: Na prática, você usaria um VectorDB com suas políticas reais.
COMMUNITY_GUIDELINES = "Regras da Comunidade: (1) Proibido Spam e links não solicitados. (2) Linguagem ofensiva ou discurso de ódio resulta em remoção imediata. (3) Comentários construtivos são encorajados. (4) Respeito mútuo é obrigatório."
tavily_tool = TavilySearch(query=COMMUNITY_GUIDELINES, max_results=2)
from tavily import TavilyClient 
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"]) # Certifique-se que a chave está no ambiente


# Configuração do Checkpoint para Persistência e HITL
conn = sqlite3.connect("moderation_checkpoints.db", check_same_thread=False)
memory = SqliteSaver(conn)

# --- Definição do Estado Compartilhado (AgentState) ---
class AgentState(TypedDict):
    """Estado compartilhado entre os agentes de moderação."""
    comentario_original: str
    politicas_relevantes: Annotated[List[str], operator.add]
    analise_do_agente: str
    status_da_moderacao: str # Ex: "PENDENTE", "APROVADO", "REMOVIDO", "EDITADO"
    justificativa_final: str
    
# --- Definição dos Prompts ---
# Os prompts definem o papel de cada agente

ANALYZER_PROMPT = """Você é um Agente Analisador de Comentários em uma plataforma de cursos. Sua tarefa é analisar o comentário de um aluno e classificá-lo.
Identifique o tom (positivo, neutro, problemático) e o tipo de problema potencial (spam, ofensivo, irrelevante).
Sua análise deve ser concisa e gerar um resultado claro que se encaixe no campo 'analise_do_agente'.

Exemplo de saída se for OK: "Comentário positivo e construtivo, sem problemas detectados."
Exemplo de saída se for ruim: "Potencialmente problemático: linguagem ofensiva e agressiva."

Comentário a analisar: {comentario}"""

REVISOR_PROMPT = """Você é o Agente Revisor. Sua tarefa é consolidar a análise do agente e as políticas encontradas para gerar uma recomendação de ação.
Recomendação deve ser uma das seguintes: "APROVAR", "REMOVER POR {{Motivo}}", ou "EDITAR POR {{Motivo}}".

Análise do Agente: {analise}
Políticas Relevantes (se houver): {politicas}

Gere apenas a recomendação de ação, sem explicações adicionais."""

# --- Definição dos Nós (Agentes) ---

def agente_analisador(state: AgentState) -> dict:
    """Nó para analisar o comentário original."""
    # Cria a string completa do prompt que será usada como HumanMessage
    full_prompt = ANALYZER_PROMPT.format(comentario=state['comentario_original'])
    
    # Invoca diretamente com a string, ou dentro de uma lista de [HumanMessage]
    response = model.invoke([HumanMessage(content=full_prompt)])

    # Define o status inicial como PENDENTE
    return {
        "analise_do_agente": response.content,
        "status_da_moderacao": "PENDENTE"
    }

class Query(BaseModel):
    query: str

def agente_pesquisador_de_politicas(state: AgentState) -> dict:
    """Nó para pesquisar políticas relevantes usando o cliente Tavily SDK."""
    
    # ... (Geração da query permanece a mesma)
    
    # Busca a query gerada DENTRO das suas políticas (Simulação de RAG)
    # Na prática, a busca real seria na internet com Tavily, ou num VectorDB.
    # Para simular RAG localmente com Tavily:
    search_query_text = f"{state['analise_do_agente']}. Regras: {COMMUNITY_GUIDELINES}"

    # Usa o cliente SDK direto, que retorna um dicionário, garantindo a estrutura
    response = tavily_client.search(query=search_query_text, max_results=2)
    
    # CORREÇÃO: Acessar a chave 'content' dos dicionários na lista 'results'
    # O resultado do SDK do Tavily é: {'query': '...', 'results': [{'content': '...', 'url': '...'}, ...]}
    politicas = [r['content'] for r in response['results']]
    
    return {"politicas_relevantes": politicas}


def agente_revisor(state: AgentState) -> dict:
    """Nó para gerar a recomendação de ação final."""
    politicas_str = "\n".join(state.get('politicas_relevantes', ["Nenhuma política relevante encontrada."]))
    
    messages = [
        SystemMessage(content=REVISOR_PROMPT.format(
            analise=state['analise_do_agente'], 
            politicas=politicas_str
        ))
    ]
    response = model.invoke(messages)
    
    # A recomendação é armazenada temporariamente na justificativa, antes da intervenção humana
    return {"justificativa_final": response.content}


def executar_acao_final(state: AgentState) -> dict:
    """Nó simulado que executa a ação e define o status final."""
    # Este nó só é atingido após a intervenção humana
    
    if "APROVAR" in state['justificativa_final'].upper():
        status = "APROVADO"
    elif "REMOVER" in state['justificativa_final'].upper():
        status = "REMOVIDO"
    else:
        status = "MANUTENÇÃO_MANUAL" # Para ações como EDITAR
        
    print(f"\n--- AÇÃO EXECUTADA ---")
    print(f"Status Final: {status}")
    print(f"Justificativa Registrada: {state['justificativa_final']}")
    print("-" * 20)
    
    return {"status_da_moderacao": status}

# --- Aresta Condicional ---
def decidir_proxima_etapa(state: AgentState):
    """Função de roteamento que decide se precisa de pesquisa ou aprova."""
    analise = state['analise_do_agente'].lower()
    
    if "problema detectado" in analise or "problemático" in analise or "ofensivo" in analise:
        return "pesquisar_politicas"
    else:
        # Se estiver OK, o revisor pode dar a aprovação final (ainda sujeito ao HITL)
        return "revisar"

# --- Construção e Compilação do Grafo ---

# 1. Adicionar o Nó final para garantir o fluxo de aprovação
# O LangGraph exige um nó de execução final para ser pausado antes dele.

builder = StateGraph(AgentState)

builder.add_node("analisar", agente_analisador)
builder.add_node("pesquisar_politicas", agente_pesquisador_de_politicas)
builder.add_node("revisar", agente_revisor)
builder.add_node("executar_acao_final", executar_acao_final)

# Ponto de entrada
builder.set_entry_point("analisar")

# Aresta Condicional: Se houver problema, pesquisa; se não, vai para o revisor
builder.add_conditional_edges(
    "analisar",
    decidir_proxima_etapa,
    {
        "pesquisar_politicas": "pesquisar_politicas",
        "revisar": "revisar"
    }
)

# Arestas normais
builder.add_edge("pesquisar_politicas", "revisar")

# O revisor sempre leva ao ponto de interrupção (Human in the Loop)
builder.add_edge("revisar", "executar_acao_final") 

# O nó final não tem saída, então ele usa END
builder.add_edge("executar_acao_final", END)

In [23]:
# Modifique a Compilação do Grafo: Pausa ANTES de 'executar_acao_final'
moderation_app = builder.compile(
    checkpointer=memory,
    interrupt_before=["executar_acao_final"]
)

# --- Cenário 1: Comentário Problemático (Exige Pesquisa) ---
def run_simple_hitl_test(comment_text):
    print(f"\n#########################################################")
    print(f"## INÍCIO DO TESTE: {comment_text[:40]}...")
    print(f"#########################################################")
    
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    
    # 1. Invocação e Pausa
    # O stream deve executar Analisador, Pesquisador e Revisor, e então pausar.
    for step in moderation_app.stream({
        "comentario_original": comment_text,
        "politicas_relevantes": [],
        "analise_do_agente": "",
        "status_da_moderacao": "",
        "justificativa_final": ""
    }, config):
        print(f"Estado Atual: {list(step.keys())[0]}")
    
    # 2. Captura da Interrupção
    
    # O agente está pausado?
    if moderation_app.get_state(config).next:
        print("\n*** FLUXO PAUSADO: Intervenção Humana Necessária ***")
        
        # Obtém o estado completo no ponto de interrupção
        current_state = moderation_app.get_state(config)
        recommendation = current_state.values['justificativa_final']
        
        print(f"Recomendação do Agente: {recommendation}")
        print(f"Próxima Ação Pendente: {current_state.next}")

        # 3. Solicita a Entrada Humana
        user_input = input("Moderador, você concorda com a recomendação e deseja APROVAR a ação? (sim/não): ").strip().lower()

        # 4. Retoma a Execução
        if user_input == 'sim':
            # Continua para o nó que estava pausado: executar_acao_final
            print("\nModerador APROVOU. Retomando o fluxo...")
            
            # O LangGraph pode ser retomado injetando o novo estado (embora aqui só precisemos da continuação)
            final_stream = moderation_app.stream(None, config)
            list(final_stream) # Executa o restante do grafo
        else:
            print("Moderador CANCELOU a aprovação. Fluxo encerrado sem ação final.")
    else:
        print("\nFluxo concluído sem intervenção. (Isso pode ocorrer se for APROVADO no roteamento).")

# Executa o teste de cenário 1
# run_simple_hitl_test("Este curso é péssimo, vou espalhar spam sobre ele em todos os canais de vocês.")

In [24]:
# --- Cenário 2: Intervenção Humana Avançada (Modificação de Estado) ---
def run_advanced_hitl_test(comment_text):
    print(f"\n#########################################################")
    print(f"## INÍCIO DO TESTE AVANÇADO: {comment_text[:40]}...")
    print(f"#########################################################")
    
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}
    
    # 1. Invocação e Pausa
    for step in moderation_app.stream({
        "comentario_original": comment_text,
        "politicas_relevantes": [],
        "analise_do_agente": "",
        "status_da_moderacao": "",
        "justificativa_final": ""
    }, config):
        print(f"Estado Atual: {list(step.keys())[0]}")
    
    # 2. Captura do Snapshot do Estado
    if moderation_app.get_state(config).next:
        print("\n*** FLUXO PAUSADO: Intervenção Humana Necessária ***")
        
        # Obtém o estado completo no ponto de interrupção
        current_state = moderation_app.get_state(config)
        recommendation = current_state.values['justificativa_final']
        
        print(f"Análise do Agente: {current_state.values['analise_do_agente']}")
        print(f"Recomendação do Agente: {recommendation}")
        
        # 3. Permite a Modificação da Resposta
        print("-" * 40)
        
        # Permite a intervenção na Justificativa
        new_justificativa = input("Moderador, insira a JUSTIFICATIVA FINAL e a AÇÃO (ex: 'REMOVER por regra X', ou deixe vazio para usar a do agente): ").strip()
        
        # 4. Injeta a Nova Informação
        
        if new_justificativa:
            final_justificativa = new_justificativa
            print(f"Justificativa do Agente SUBSTITUÍDA pela do Moderador.")
        else:
            final_justificativa = recommendation
            print(f"Usando a Justificativa Original do Agente: {recommendation}")
        
        # 5. Atualiza o Estado do Grafo
        
        # Criamos o novo estado a ser injetado (apenas a chave modificada)
        new_values = {"justificativa_final": final_justificativa}
        
        # Usamos update_state para injetar a nova informação
        moderation_app.update_state(config, new_values)
        
        # 6. Finaliza o Fluxo
        print("\nIntervenção registrada. Retomando o fluxo...")
        final_stream = moderation_app.stream(None, config)
        list(final_stream) # Executa o nó executar_acao_final e o END
    else:
        print("\nFluxo concluído sem intervenção.")

# Executa o teste de cenário 2
# run_advanced_hitl_test("Este é um comentário útil e construtivo, muito bom!")
run_advanced_hitl_test("Pessoal, sigam este link externo [link-de-spam] para um curso mais barato!")


#########################################################
## INÍCIO DO TESTE AVANÇADO: Pessoal, sigam este link externo [link-d...
#########################################################
Estado Atual: analisar
Estado Atual: pesquisar_politicas


KeyError: 'Motivo'

In [5]:
from typing import TypedDict, List

class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

In [7]:
PLAN_PROMPT = """Você é um escritor especialista com a tarefa de criar um esboço de alto nível para uma redação. \
Escreva esse esboço para o tópico fornecido pelo usuário. Apresente um plano da redação junto com quaisquer notas \
ou instruções relevantes para as seções."""

In [8]:
WRITER_PROMPT = """Você é um assistente de redação com a tarefa de escrever excelentes redações de 5 parágrafos. \
Gere a melhor redação possível para a solicitação do usuário e o esboço inicial. \
Se o usuário fornecer críticas, responda com uma versão revisada das suas tentativas anteriores. \
Utilize todas as informações abaixo conforme necessário:

------

{content}"""

In [9]:
REFLECTION_PROMPT = """Você é um professor corrigindo uma redação submetida. \
Gere uma crítica e recomendações para a submissão do usuário. \
Forneça recomendações detalhadas, incluindo pedidos sobre extensão, profundidade, estilo, etc."""


In [10]:
RESEARCH_PLAN_PROMPT = """Você é um pesquisador encarregado de fornecer informações que podem \
ser usadas ao escrever a seguinte redação. Gere uma lista de consultas de pesquisa que \
recolham quaisquer informações relevantes. Gere no máximo 3 consultas."""


In [11]:
RESEARCH_CRITIQUE_PROMPT = """Você é um pesquisador encarregado de fornecer informações que podem \
ser usadas ao fazer quaisquer revisões solicitadas (conforme descrito abaixo). \
Gere uma lista de consultas de pesquisa que recolham quaisquer informações relevantes. Gere no máximo 3 consultas."""


In [ ]:
from pydantic import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [13]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [14]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [15]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [16]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [17]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [18]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [19]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [20]:
builder = StateGraph(AgentState)

In [21]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [22]:
builder.set_entry_point("planner")

In [23]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [24]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [25]:
graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display, Markdown
import os

print("\n--- Tentando Gerar PNG do Grafo via Mermaid (Requer Playwright!) ---")
try:

    image_data = graph.get_graph().draw_mermaid_png()
    display(Image(data=image_data))
    print("Grafo PNG gerado e exibido com sucesso!")

except Exception as e:

    print(f"\nErro ao tentar gerar o PNG do grafo: {e}")
    print("Isso pode ocorrer por:")
    print("1. O método `.draw_mermaid_png()` não existe na sua versão do LangGraph.")
    print("2. Faltam dependências como 'playwright' ou seus drivers não foram instalados.")
    print("   Tente: pip install playwright && playwright install")
    print("3. Outro erro inesperado ao acessar o grafo ou renderizar.")

    print("\n--- Tentando gerar apenas o código Mermaid (Fallback) ---")
    try:

        mermaid_code = graph.get_graph().draw_mermaid()
        print("\n--- Código Mermaid Gerado (Cole em https://mermaid.live/ ou use Markdown no notebook) ---")
        print(mermaid_code)

    except Exception as e_mermaid:
        print(f"Erro ao gerar código Mermaid: {e_mermaid}")
        print("Verifique se 'graph.get_graph()' está correto e acessível.")

In [31]:
from tavily import TavilyClient

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    "task": "Qual é a diferença entre o langchain e langsmith",
    "max_revisions": 2,
    "revision_number": 1,
    "content": [],
}, thread):
    print(s)